In [1]:
# İmport necessary libraries
from datetime import *
from dateutil.relativedelta import *
import pandas as pd
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
from scipy import optimize
import numpy as np
import numpy.polynomial.polynomial as pol
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv("daily_data.csv")
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [3]:
T = pd.read_csv("T_column.csv")

In [4]:
df["trd_exctn_dt"] = pd.to_datetime(df["trd_exctn_dt"])
df["MATURITY"] = pd.to_datetime(df.MATURITY, format="%Y%m%d")
df["PRINCIPAL_AMT"] = df["PRINCIPAL_AMT"] / 10
df["PRINCIPAL_AMT"] = df["PRINCIPAL_AMT"].astype("int")

In [5]:
# Convert INTEREST_FREQUENCY column to appropriate format.
df["INTEREST_FREQUENCY"] = df["INTEREST_FREQUENCY"].astype("int")
df["INTEREST_FREQUENCY"] = df["INTEREST_FREQUENCY"].astype("str")
df["INTEREST_FREQUENCYforDates"] = df["INTEREST_FREQUENCY"].replace({"2": "6", "4": "3", "1": "12", "12": "1"})
df["INTEREST_FREQUENCY"] = df["INTEREST_FREQUENCY"].astype("int")
df["INTEREST_FREQUENCYforDates"] = df["INTEREST_FREQUENCYforDates"].astype("int")

In [6]:
df["INTEREST_FREQUENCYforDates"] 

0        6
1        6
2        6
3        6
4        6
        ..
56805    6
56806    6
56807    6
56808    6
56809    6
Name: INTEREST_FREQUENCYforDates, Length: 56810, dtype: int32

In [7]:
# Set zero coupons first interest date to -999.
zeros = df[df["FIRST_INTEREST_DATE"].isna()]
zeros_FIRST_INTEREST_DATE = zeros["FIRST_INTEREST_DATE"].values
zeros["FIRST_INTEREST_DATE"] = -999

In [8]:
df = df[df["FIRST_INTEREST_DATE"] > 0]
df["FIRST_INTEREST_DATE"] = df["FIRST_INTEREST_DATE"].astype("int")
df["FIRST_INTEREST_DATE"] = pd.to_datetime(df.FIRST_INTEREST_DATE, format="%Y%m%d")

df["LAST_INTEREST_DATE"] = df["LAST_INTEREST_DATE"].astype("int")
df["LAST_INTEREST_DATE"] = pd.to_datetime(df.LAST_INTEREST_DATE, format="%Y%m%d")

In [9]:
df = pd.concat([df, zeros], axis=0)

In [10]:
# Sort Values by firstly cusip_id then exctn day - similar to groupby
sortlist = ["cusip_id", "trd_exctn_dt"]
df = df.sort_values(sortlist)

In [11]:
df["stlmnt_dt"] = pd.to_datetime(df.stlmnt_dt, format="%Y%m%d")

In [12]:
df["T"] = T["T"]

In [13]:
# FIRST_INTEREST_DATE[i], MATURITY[i], trd_exctn_dt[i], FreqForDates[i]
# Function that calculates year.
def CalculateYear(row):
    
    FirstPeriod = 0
    start = row["FIRST_INTEREST_DATE"]
    end = row["MATURITY"]
    exctn = row["trd_exctn_dt"]
    settlement = row["stlmnt_dt"]
    freq = row["INTEREST_FREQUENCYforDates"]

    if start == -999:
        delta = relativedelta(end, exctn)
        n = delta.years + delta.months / 12 + delta.days / 360
        choosenpaid = "Not Calculated"
        FirstPaid = 0

    else:
        # Create schedule
        paid = []
        paid.append(start)
        x = start
        int(freq)
        while x < end:
            # print(type(freq))
            x = x + relativedelta(months=+(freq))
            paid.append(x)

        # Select valid schedule after execution date. Starting from first paid date until Last paid Date.
        res = next(x for x, val in enumerate(paid) if val > exctn)

        choosenpaid = paid[res:]

        # First interest day after exctn date.

        a = [n for n, i in enumerate(paid) if i > exctn][0]

        b = paid[a : a + 1]

        if b[0] == end:
            delta = relativedelta(end, exctn)
            n = delta.years + delta.months / 12 + delta.days / 360

        # Find how many years between first payment after execution date and last payment.

        # Get the relativedelta between two dates ( First interest payment day after execution date = choosenpaid[0])))
        # last interest payment date
        elif a != end:
            delta = relativedelta(choosenpaid[-1], choosenpaid[0])
            n = delta.years + delta.months / 12 + delta.days / 360

    # First Period şu anda Her zaman Bu şekilde hesaplanıyor ama muhtemelen FirstPeriod için de bir condition Gerekli.
    # Eğer Settlement - Maturity bu hesapladığımız FirstPeriod a eşitse ?? benzer bi durum.

    if choosenpaid[-1] > end:
        # print('asd')
        choosenpaid.pop()
        delta = relativedelta(choosenpaid[-1], choosenpaid[0])
        n = delta.years + delta.months / 12 + delta.days / 360

    if start != -999:

        First = relativedelta(choosenpaid[0], settlement)
        FirstPeriod = First.years + First.months / 12 + First.days / 360
        # print(FirstPeriod)

        if FirstPeriod < 0:
            First = relativedelta(choosenpaid[1], settlement)
            FirstPeriod = First.years + First.months / 12 + First.days / 360
            choosenpaid = choosenpaid[1:]
            n = n - freq / 12

    

    return n, choosenpaid, FirstPeriod

In [14]:
def CalculateYearUsage(row):
    
    end = row["MATURITY"]
    exctn = row["trd_exctn_dt"]

    if row["FIRST_INTEREST_DATE"] != -999:

        if len(CalculateYear(row)[1]) == 1:
            T = CalculateYear(row)[2]

        elif len(CalculateYear(row)[1]) > 1:
            T = CalculateYear(row)[2] + CalculateYear(row)[0]

    elif row["FIRST_INTEREST_DATE"] == -999:

        delta = relativedelta(end, exctn)
        T = delta.years + delta.months / 12 + delta.days / 360

    return T

In [15]:
# Calculating T with Different Conditions.
from tqdm.notebook import tqdm
tqdm.pandas()
df["Real_T"] = df.progress_apply(lambda row: CalculateYearUsage(row), axis=1)

  0%|          | 0/56810 [00:00<?, ?it/s]

In [16]:
df = df.loc[df["Real_T"] < 50]

In [18]:
def ytm(row):
    # Get Calculated T.

    T= row['Real_T']

    # For Not Zero Coupon Bonds

    if (row['FIRST_INTEREST_DATE'] != -999) :

        periods = T*row['INTEREST_FREQUENCY']            
        C = row['COUPON']/100*row['PRINCIPAL_AMT'] / row['INTEREST_FREQUENCY']            
        F = row['INTEREST_FREQUENCY']

        # If mod operation equals to 0, it means that bond have integer T similar to 3, 5, 11  
        # That means payments happened in same- equal periods.

        if (row['Real_T'] % (row['INTEREST_FREQUENCYforDates'] / 12) == 0 ): 
            # Tam Sayılar için
            periods= int(periods)

            dt = [ (i+row['INTEREST_FREQUENCY']) / row['INTEREST_FREQUENCY'] for i in range(int(periods))]
            #print(dt)

            if (dt[0] != row['INTEREST_FREQUENCYforDates'] / 12):
                dt.insert(0,row['INTEREST_FREQUENCYforDates'] / 12)

            if dt[-1] != T:
                dt = dt[:-1]

            if dt[-1] != T:
                dt.pop()

            if dt[-1] != T:
                dt.pop() 

            if dt[-1] !=T:
                print(dt,T)

            # Control Statements
            ytm_funcc = lambda y : sum ([C/(1+y/row['INTEREST_FREQUENCY'])**(row['INTEREST_FREQUENCY']*T) for h in dt]) + row['PRINCIPAL_AMT']/(1+y/row['INTEREST_FREQUENCY'])**(row['INTEREST_FREQUENCY']*row['T']) - row['dirty_price']
            a=optimize.newton(ytm_funcc, 0.05)

            return a
        
        # If mod operation is not equals to 0. it means years until first payment is not equal with other payment periods.
        # It's gonna add the years until first payment date to list. Then it will continue to add until last payment with equal periods.
        elif (row['Real_T'] % (row['INTEREST_FREQUENCYforDates'] / 12) != 0 ):
            periods = int(periods)
            dt = []
            First = CalculateYear(row)[2]
            # Adding First- Years until first payment 
            dt.append(First) # T den Gelen Olay Eklenecek.               

            # Adding rest of them until last payment.
            for i in range(int(periods)):       
                    k = First + ((i+1) / row['INTEREST_FREQUENCY'])
                    dt.append(k)
                    
            if (T != dt[-1]):
                dt = dt[:-1]

            #dt means created schedule 

            ytm_func = lambda y : sum ([ C / (1+ y/row['INTEREST_FREQUENCY'])**(row['INTEREST_FREQUENCY']*T) for k in dt]) + row['PRINCIPAL_AMT']/(1+y/row['INTEREST_FREQUENCY'])**(T) - row['dirty_price']
            a=optimize.newton(ytm_func, 0.05)
            return a 

    # For 0 bond Coupons. 
    elif row['FIRST_INTEREST_DATE'] == -999:
        ytm_funcx = lambda y : row['PRINCIPAL_AMT'] / (1+y)**(row['Real_T']) - row['daily_clean_pr']
        return optimize.newton(ytm_funcx, 0.05)

In [19]:
from tqdm.notebook import tqdm
tqdm.pandas()
df["ytm"] = df.progress_apply(lambda row: ytm(row), axis=1)

  0%|          | 0/56211 [00:00<?, ?it/s]

In [20]:
def Payment_Years_Calculator(row):
    
    T = row["Real_T"]
    
    # For Not Zero Coupon Bonds
    if row["FIRST_INTEREST_DATE"] != -999:

        periods = T * row["INTEREST_FREQUENCY"]
        C = row["COUPON"] / 100 * row["PRINCIPAL_AMT"] / row["INTEREST_FREQUENCY"]
        F = row["INTEREST_FREQUENCY"]
        if row["Real_T"] % (row["INTEREST_FREQUENCYforDates"] / 12) == 0:
            # Tam Sayılar için
            periods = int(periods)

            dt = [
                (i + row["INTEREST_FREQUENCY"]) / row["INTEREST_FREQUENCY"]
                for i in range(int(periods))
            ]
            # print(dt)

            if dt[0] != row["INTEREST_FREQUENCYforDates"] / 12:
                dt.insert(0, row["INTEREST_FREQUENCYforDates"] / 12)

            if dt[-1] != T:
                dt = dt[:-1]

            if dt[-1] != T:
                dt.pop()

            if dt[-1] != T:
                dt.pop()

            if dt[-1] != T:
                print(dt, T)

            # Control Statements
            return dt, C

        # If mod operation is not equals to 0. it means years until first payment is not equal with other payment periods.
        # It's gonna add the years until first payment date to list. Then it will continue to add until last payment with equal periods.
        elif row["Real_T"] % (row["INTEREST_FREQUENCYforDates"] / 12) != 0:
            periods = int(periods)
            dt = []
            First = CalculateYear(row)[2]
            # Adding First- Years until first payment
            dt.append(First)  # T den Gelen Olay Eklenecek.

            # Adding rest of them until last payment.
            for i in range(int(periods)):
                k = First + ((i + 1) / row["INTEREST_FREQUENCY"])
                dt.append(k)

            if T != dt[-1]:
                dt = dt[:-1]

            return dt, C

    # For 0 bond Coupons.
    elif row["FIRST_INTEREST_DATE"] == -999:

        return -1, -1

In [21]:
def Duration(row):
    # First Version was like this: 
    
    # zf = pd.DataFrame(data = Payment_Years)
    # zf.columns = ['Payment_Dates']
    # zf['Cash_Flow'] = CashFlow
    # zf['ytm'] = row['ytm']
    # zf['Weights']  = zf['Present_Val'] / zf['Present_Val'].sum()
    # W = zf['Weights'].sum()
    # zf['Weights*Payment_Dates'] = (zf['Present_Val'] / zf['Present_Val'].sum()) * zf['Payment_Dates']
    # zf['Weights*Payment_Dates'].sum(), W
        
    # I've changed this for faster calculation.
    
    if Payment_Years_Calculator(row)[0] == -1:

        return row["Real_T"], row["Real_T"]

    else:

        L = len(Payment_Years_Calculator(row)[0])
        Payment_Years = Payment_Years_Calculator(row)[0]
        Payment_Years.append(Payment_Years[-1])

        CashFlow = L * [Payment_Years_Calculator(row)[1]]
        CashFlow.append(100)

        F = row["INTEREST_FREQUENCY"]
        K = L + 1
        ytm = row["ytm"]
        ytm = K * [ytm]
        FList = K * [F]
        Present_Val = np.divide(
            CashFlow, (1 + np.divide(ytm, FList)) ** (np.multiply(Payment_Years, FList))
        )

        Weights = np.divide(Present_Val, Present_Val.sum())
        W = Weights.sum()
        Duration = np.multiply(Weights, Payment_Years)
        D = Duration.sum()
        return D, W

In [22]:
from tqdm.notebook import tqdm
tqdm.pandas()
df['Duration'] = df.progress_apply(lambda row: Duration(row)[0], axis = 1)

  0%|          | 0/56211 [00:00<?, ?it/s]

In [23]:
df['Modified_Duration'] = df['Duration'] /( 1 + (df['ytm'] / df['INTEREST_FREQUENCY']))

In [24]:
# Functions for calculating zero bonds and others bonds prices without any assumptions.
def zero_bond_price(row,y):
    a = row['PRINCIPAL_AMT']/(1+y)**(row['Real_T']) 
    return a    

def bond_price(row,y):
    if  row['FIRST_INTEREST_DATE'] != -999:
        dt = Payment_Years_Calculator(row)[0] # Periods are not same 
        T = row['Real_T']
        periods = T*row['INTEREST_FREQUENCY']            
        C = row['COUPON']/100*row['PRINCIPAL_AMT'] / row['INTEREST_FREQUENCY']            
        F = row['INTEREST_FREQUENCY']
        a = sum ([ C / (1+ y/row['INTEREST_FREQUENCY'])**(row['INTEREST_FREQUENCY']*T) for k in dt]) + row['PRINCIPAL_AMT']/(1+y/row['INTEREST_FREQUENCY'])**(row['Real_T']) 
        return a
    # Pricing 0 coupon bonds.
    elif row['FIRST_INTEREST_DATE'] == -999:
        return zero_bond_price(row,y)      

In [25]:
def bond_convexity(row):
    
    ytm = row["ytm"]
    
    A = 0.01
    price = row["dirty_price"]

    ytm_minus = ytm - 0.01
    price_minus = bond_price(row, ytm_minus)

    ytm_plus = ytm + 0.01
    price_plus = bond_price(row, ytm_plus)

    convexity = (price_minus + price_plus - 2 * price) / (price * A**2)
    return convexity / 100

In [26]:
df["Bond_Convexity"] = df.progress_apply(lambda row: bond_convexity(row), axis=1)

  0%|          | 0/56211 [00:00<?, ?it/s]

In [27]:
def BusinesDay(Year, Month, Day):
    a = datetime(Year, Month, 1)
    if Month == 12:
        b = datetime(Year + 1, 1, 1)
    else:
        b = datetime(Year, Month + 1, 1)

    BdayList = pd.date_range(a, b, freq="B")[-6:].day.tolist()

    # Get Business Day of Month and return if its last 5 business days.

    if any(item == 1 for item in BdayList):
        BdayList.remove(1)
        if len(BdayList) == 6:
            BdayList = BdayList[-5:]

    if len(BdayList) == 6:
        BdayList = BdayList[-5:]

    if any(item == Day for item in BdayList):
        return True
    else:
        return False

In [28]:
def BusinesDayFirst5(Year, Month, Day):
    a = datetime(Year, Month, 1)
    if Month == 12:
        b = datetime(Year + 1, 1, 1)
    else:
        b = datetime(Year, Month + 1, 1)
    BdayList = pd.date_range(a, b, freq="B")[0:5].day.tolist()
    # Get Business Day of Month and return if its last 5 business days.
    if any(item == Day for item in BdayList):
        return True
    else:
        return False

In [29]:
zf = pd.DataFrame()
from tqdm import tqdm
for m in tqdm(df["cusip_id"].unique()):
    Selected = pd.DataFrame()
    SelectFromQF = pd.DataFrame()
    qf = df[df['cusip_id']==m]
    qf = qf.reset_index()
    qf.drop(['index'],inplace=True,axis=1) 
    UniqueYears = qf['trd_exctn_dt'].dt.year.unique()
    
    for m in range(1,13):
        for j in range(len(UniqueYears)):
            Yrs = qf['trd_exctn_dt'].dt.year.unique()[j]

            tf = qf[(qf['trd_exctn_dt'].dt.month == m) & (qf['trd_exctn_dt'].dt.year == Yrs) ][['trd_exctn_dt']] # 3,4 yerine i i +1 
            yf = qf[(qf['trd_exctn_dt'].dt.month == m) & (qf['trd_exctn_dt'].dt.year == Yrs) ][['trd_exctn_dt']]
           
            YearList_Next = yf['trd_exctn_dt'].dt.year.tolist()
            MonthList_Next = yf['trd_exctn_dt'].dt.month.tolist()
            DayList_Next = yf['trd_exctn_dt'].dt.day.tolist()

            YearList = tf['trd_exctn_dt'].dt.year.tolist()
            MonthList = tf['trd_exctn_dt'].dt.month.tolist()
            DayList = tf['trd_exctn_dt'].dt.day.tolist()

            # Get Months Days into this list:
            GetDays = []
            GetMonths = []
            GetYears = []

           
            # Gün kadar Da yıl olması lazım        
            ChoosenYears = []
            ChoosenDays = []
            ChoosenMonths = []

            ChoosenYears_Next = []
            ChoosenDays_Next = []
            ChoosenMonths_Next = []
            a = 0
            b = 0
            for i in range(len(DayList)):
                if BusinesDay(YearList[i], MonthList[i], DayList[i]) == 1: 

                    # Business Day olanları Bir Listeye al -sırala - ensonuncuyu seç
                    a = a + 1              
                    # Add Choosen Days From That Month
                    ChoosenYears.append(YearList[i])
                    ChoosenDays.append(DayList[i])
                    ChoosenMonths.append(MonthList[i])                


                elif BusinesDay(YearList[i], MonthList[i], DayList[i]) == 0 :  
                    for j in range(len(DayList_Next)):
                        if BusinesDayFirst5(YearList_Next[j], MonthList_Next[j], DayList_Next[j]) == 1:

                            b = b+1         
                            # Add Choosen Days From That Month
                            ChoosenYears_Next.append(YearList_Next[j])
                            ChoosenDays_Next.append(DayList_Next[j])
                            ChoosenMonths_Next.append(MonthList_Next[j])
                        else:
                            continue
            if a!=0:             
                day = sorted(ChoosenDays)[-1]
                month = ChoosenMonths[0]
                year = ChoosenYears[0]

                GetDays.append(day)
                GetMonths.append(month)
                GetYears.append(year)

            if b!=0:            
                day_next = sorted(ChoosenDays_Next)[0]
                month_next = ChoosenMonths_Next[0]
                Year_next = ChoosenYears_Next[0]

                GetDays.append(day_next)
                GetMonths.append(month_next)
                GetYears.append(Year_next)


            if len(GetMonths)==0:
                GetMonths.append(0)
            if len(GetDays)==0:
                GetDays.append(0)
            if len(GetYears)==0:
                GetYears.append(0) 


            SelectedDF = pd.DataFrame(data = GetMonths, columns=['Month']) 
            SelectedDF['Days'] = GetDays
            SelectedDF['Years'] = GetYears 

            SelectedDF.drop_duplicates()
            Selected= pd.concat([Selected,SelectedDF],axis=0)
            Selected = Selected.drop_duplicates()

    Selected = Selected[(Selected != 0).all(1)]
    
    qfMonth = Selected['Month'].values
    qfDay = Selected['Days'].values
    qfYear = Selected['Years'].values

    Sort = ['Years','Month','Days']
    Selected = Selected.sort_values(Sort)

    arr = np.arange(0,Selected.shape[0])
    Selected.set_index(arr,inplace=True)
    
    LastCleanDays = []
    LastCleanMonths = []
    LastCleanYears = []

    indexlist = []
    for i in range(Selected.shape[0]-1):    
        D, M, Y = Selected.Days.iloc[i],  Selected.Month.iloc[i] , Selected.Years.iloc[i] 
        D_2, M_2, Y_2 = Selected.iloc[i+1].Days,Selected.iloc[i+1].Month,Selected.iloc[i+1].Years
        if (BusinesDay(Y, M ,D) == 1) and (M +1  == M_2 or (M-1 == M_2 and Y_2 - 1 )) and BusinesDayFirst5(Y_2, M_2, D_2):
            indexlist.append(i+1)

    # Select Valid Dates From Unique Cusisp_id
    for i in range (len(qfMonth)):
        sf = qf[(qf['trd_exctn_dt'].dt.year == qfYear[i]) & (qf['trd_exctn_dt'].dt.month == qfMonth[i]) & (qf['trd_exctn_dt'].dt.day == qfDay[i])]
        SelectFromQF = pd.concat([SelectFromQF,sf],axis=0)                

    if SelectFromQF.shape[0] > 0: 
        # Sort Values, reset index after selecting.
        SelectFromQF = SelectFromQF.sort_values('trd_exctn_dt')
        SelectFromQF = SelectFromQF.reset_index()

    
    if Selected.shape[0] > 1:
        # Check if first row is from first 5 business day of a month and second row from last 5 business day of a month.
        # And if they're from same month, drop first row.
        a,b,c = Selected.Years.iloc[0],  Selected.Month.iloc[0], Selected.Days.iloc[0]
        d,e,f = Selected.Years.iloc[1],  Selected.Month.iloc[1], Selected.Days.iloc[1]
        if  BusinesDayFirst5(a,b,c) and BusinesDay(d,e,f) and b == e:    
            indexlist.insert(0,0)

    # Drop unvalid Dates.
    bad_df = SelectFromQF.index.isin(indexlist)
    SelectFromQF = SelectFromQF[~bad_df]


    if SelectFromQF.shape[0] > 0: 
        # Sort Values, reset index after selecting.
        SelectFromQF = SelectFromQF.sort_values('trd_exctn_dt')
        SelectFromQF = SelectFromQF.reset_index()

    zf = pd.concat([zf, SelectFromQF], axis=0)

100%|████████████████████████████████████████████████████████████████████████████████| 945/945 [04:52<00:00,  3.23it/s]


In [30]:
zf

,level_0,index,cusip_id,trd_exctn_dt,rptd_pr,entrd_vol_qt,stlmnt_dt,yld_pt,ISSUE_ID,ISSUER_ID,ISSUER_CUSIP,CUSIP_NAME,PREFERRED_SECURITY,DATED_DATE,PRINCIPAL_AMT,COUPON,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,INTEREST_FREQUENCY,MATURITY,AGENT_ID,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,COUNTRY,daily_clean_pr,day_count,AI,dirty_price,INTEREST_FREQUENCYforDates,T,Real_T,ytm,Duration,Modified_Duration,Bond_Convexity
0,0,0,00138GAA7,2019-01-28,130.559,5000.0,2019-01-30,4.961619,597773,44437,00138G,AIG LIFE HLDGS INC,NaN,20130711.0,100,8.500,2014-01-01 00:00:00,2030-01-01 00:00:00,2,2030-07-01,64505,2.0,23,187.0,524113.0,USA,129.359000,29,0.684722,130.043722,6,11.583333,11.419444,0.050967,7.991284,7.792698,0.742892
1,1,1,00138GAA7,2019-03-05,129.777,30000.0,2019-03-07,5.017889,597773,44437,00138G,AIG LIFE HLDGS INC,NaN,20130711.0,100,8.500,2014-01-01 00:00:00,2030-01-01 00:00:00,2,2030-07-01,64505,2.0,23,187.0,524113.0,USA,129.697000,66,1.558333,131.255333,6,11.483333,11.316667,0.050255,7.898989,7.705371,0.731918
2,2,3,00138GAA7,2019-04-03,131.692,22000.0,2019-04-05,4.814338,597773,44437,00138G,AIG LIFE HLDGS INC,NaN,20130711.0,100,8.500,2014-01-01 00:00:00,2030-01-01 00:00:00,2,2030-07-01,64505,2.0,23,187.0,524113.0,USA,131.700473,94,2.219444,133.919917,6,11.402778,11.238889,0.048038,7.853784,7.669566,0.726922
3,3,6,00138GAA7,2019-05-07,131.985,5000.0,2019-05-09,4.766097,597773,44437,00138G,AIG LIFE HLDGS INC,NaN,20130711.0,100,8.500,2014-01-01 00:00:00,2030-01-01 00:00:00,2,2030-07-01,64505,2.0,23,187.0,524113.0,USA,131.985000,128,3.022222,135.007222,6,11.308333,11.144444,0.047410,7.768540,7.588649,0.716774
4,4,8,00138GAA7,2019-06-24,136.442,2000.0,2019-06-26,4.309204,597773,44437,00138G,AIG LIFE HLDGS INC,NaN,20130711.0,100,8.500,2014-01-01 00:00:00,2030-01-01 00:00:00,2,2030-07-01,64505,2.0,23,187.0,524113.0,USA,136.475333,175,4.131944,140.607278,6,11.175000,11.013889,0.042712,7.706455,7.545316,0.710000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,11,50,989822AA9,2019-07-29,115.736,5000.0,2019-07-31,3.103203,24074,4613,989822,ZURICH REINSURANCE CENTRE HLDGS INC,N,19931020.0,100,7.125,1994-04-15 00:00:00,2023-04-15 00:00:00,2,2023-10-15,4613,2.0,23,72799.0,524126.0,USA,115.736000,105,2.078125,117.814125,6,4.269444,4.208333,0.044441,3.670095,3.590317,0.083482
7,13,57,989822AA9,2019-08-28,116.857,500000.0,2019-08-30,2.771046,24074,4613,989822,ZURICH REINSURANCE CENTRE HLDGS INC,N,19931020.0,100,7.125,1994-04-15 00:00:00,2023-04-15 00:00:00,2,2023-10-15,4613,2.0,23,72799.0,524126.0,USA,116.727167,135,2.671875,119.399042,6,4.186111,4.125000,0.039956,3.592260,3.521899,0.081056
8,15,63,989822AA9,2019-09-25,115.798,10000.0,2019-09-27,2.956699,24074,4613,989822,ZURICH REINSURANCE CENTRE HLDGS INC,N,19931020.0,100,7.125,1994-04-15 00:00:00,2023-04-15 00:00:00,2,2023-10-15,4613,2.0,23,72799.0,524126.0,USA,116.669290,162,3.206250,119.875540,6,4.108333,4.050000,0.039070,3.518341,3.450927,0.078493
9,17,71,989822AA9,2019-10-29,116.392,10000.0,2019-10-31,2.724944,24074,4613,989822,ZURICH REINSURANCE CENTRE HLDGS INC,N,19931020.0,100,7.125,1994-04-15 00:00:00,2023-04-15 00:00:00,2,2023-10-15,4613,2.0,23,72799.0,524126.0,USA,116.642958,15,0.296875,116.939833,6,4.013889,3.958333,0.039583,3.536737,3.468098,0.072758
